<a href="https://colab.research.google.com/github/LC1332/Prophet-Andrew-Ng/blob/main/prophet-code/prophet-retrieve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 骆驼先知

骆驼先知是 李鲁鲁 受到吴恩达的prompt Engineering的启发

制作的一个纪伯伦的《先知》的拓展版本

运行这个notebook你需要OpenAI的API Token

项目链接 [https://github.com/LC1332/Prophet-Andrew-Ng](https://github.com/LC1332/Prophet-Andrew-Ng)

骆驼先知是[Luotuo(骆驼)](https://github.com/LC1332/Luotuo-Chinese-LLM)的子项目之一，后者由李鲁鲁，冷子昂，陈启源发起。

## retrieve版本的说明

相比于Random的版本，retrieve会根据你的query_topic

使用LuotuoBERT去搜索最接近的文本，然后进行In Context的few shot Learning

在colab运行时，我们建议使用GPU来进行运行。

In [ ]:
! pip install openai

In [2]:
import openai
import os

# 导入第三方库

openai.api_key  = 'sk-'
# 李鲁鲁 在这里设置你的API_KEY

In [29]:
!rm -r -f /content/Prophet-Andrew-Ng/
#从项目中获取数据
!git clone https://github.com/LC1332/Prophet-Andrew-Ng

Cloning into 'Prophet-Andrew-Ng'...
remote: Enumerating objects: 241, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 241 (delta 21), reused 27 (delta 9), pack-reused 189
Receiving objects: 100% (241/241), 673.79 KiB | 7.74 MiB/s, done.
Resolving deltas: 100% (119/119), done.


## 数据读取

读取prompt-data中的文本数据，作为in-context-learning的数据

In [30]:
# 如果你使用本地的版本，你的路径应该为
# prophet_data_folder = './../prophet-data'

# 在这里我们考虑colab的版本
prophet_data_folder = '/content/Prophet-Andrew-Ng/prophet-data'

import os

titles = []
title_to_text = {}

# scan all txt file in prophet_data_folder
for file in os.listdir(prophet_data_folder):
    if file.endswith('.txt'):
        title_name = file[:-4]
        titles.append(title_name)

        with open(os.path.join(prophet_data_folder, file), 'r') as f:
            title_to_text[title_name] = f.read()

# report length of each text
for title in titles:
    print(title, len(title_to_text[title]))

自由 781
法律 640
爱 789
婚姻 292
劳作 935
孩子 315
痛苦 289


我们将在后续课程中深入探究 OpenAI 提供的 ChatCompletion API 的使用方法，在此处，我们先将它封装成一个函数，你无需知道其内部机理，仅需知道调用该函数输入 Prompt 其将会给出对应的 Completion 即可。

In [31]:
# 一个封装 OpenAI 接口的函数，参数为 Prompt，返回对应结果
def get_completion(prompt, model="gpt-3.5-turbo"):
    '''
    prompt: 对应的提示
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
    '''
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # 模型输出的温度系数，控制输出的随机程度
    )
    # 调用 OpenAI 的 ChatCompletion 接口
    return response.choices[0].message["content"]


在这里我们先假设一个query的主题 "时间"

In [6]:
query_topic = "时间"

定义先知和市民的身份，这里是为了方便后面的泛化

In [7]:
prophet_name = "先知"

citizen_name = "市民"

形成一个shot的句子输出

In [32]:
# 实现一个Python函数 ensemble_one_shot ，输入为prophet_name, citizen_name, sample_topic，sample_answer， 输出为一个组织好的字符串
# 例子输入: 先知, 市民, 衣服， "你们的衣服遮掩了你们许多的美，却不能遮盖住丑。 \n 尽管你们借衣服寻求隐私的自由，但你们找到的却是羁绊和束缚。"
# 例子输出: """
# <市民>:请和我们讨论一下"衣服"

# <先知>: 你们的衣服遮掩了你们许多的美，却不能遮盖住丑。
# 尽管你们借衣服寻求隐私的自由，但你们找到的却是羁绊和束缚。
# """
def ensemble_one_shot(prophet_name, citizen_name, sample_topic, sample_answer):
    # 组织对话文本
    dialogue = "<{}>:```请和我们讨论一下\"{}\"```\n\n<{}>:```{}```\n\n".format(citizen_name, sample_topic, prophet_name, sample_answer)
    return dialogue

# unit test for ensemble_one_shot
prophet_name = "先知"
citizen_name = "市民"
sample_topic = "衣服"
sample_answer = "你们的衣服遮掩了你们许多的美，却不能遮盖住丑。\n尽管你们借衣服寻求隐私的自由，但你们找到的却是羁绊和束缚。"

dialogue = ensemble_one_shot(prophet_name, citizen_name, sample_topic, sample_answer)
print(dialogue)

<市民>:```请和我们讨论一下"衣服"```

<先知>:```你们的衣服遮掩了你们许多的美，却不能遮盖住丑。
尽管你们借衣服寻求隐私的自由，但你们找到的却是羁绊和束缚。```




下面我们来组织完整的prompt，我们假设已经选取了两个主题作为例子 孩子和爱

In [33]:
selected_sample = ["孩子","爱"]

def organize_prompt( query_topic, selected_sample ):
    prompt = """你的任务是以纪伯伦中《先知》的语言风格回答问题。\
    先知会使用大量的比喻修辞手法。感情色彩浓厚，富有感染力和感召力，具有启迪性和哲理性。\
    语言风格简洁明了，比喻和象征性的意象丰富多彩，既充满了哲理性和哲学深度，又富有感人肺腑的情感色彩。\n\n"""
    
    for sample_topic in selected_sample:
        # find sample_answer in dictionary
        sample_answer = title_to_text[sample_topic]
        prompt += ensemble_one_shot(prophet_name, citizen_name, sample_topic, sample_answer)

    prompt += """<{}>:```请和我们讨论一下"{}"```\n\n""".format(citizen_name, query_topic)

    return prompt

# write a unit test for organize_prompt
# query_topic = "时间"
# selected_sample = ["孩子","爱"]
# prompt = organize_prompt( query_topic, selected_sample )
# print(prompt)

在第一个版本中，我们selected_sample先使用随机的策略

在后面的版本中我准备引入LuotuoBERT 去选取更接近主题的问题

In [34]:
import random

# def function random_select_title, random pick k sample from titles
def random_select_title(titles, k):
    return random.sample(titles, k)

# unit test for random_select_title
selected_sample = random_select_title(titles, 2)
print(selected_sample)

['孩子', '劳作']


## 升级randome_select到retrieve topics

这里我们开始引入LuotuoBERT，相应的项目见 [骆驼嵌入](https://github.com/LC1332/Luotuo-Text-Embedding)

我们需要先安装hugging face的代码库

In [ ]:
!pip install transformers

然后从hugging face上，载入对应的模型

In [35]:
import torch
from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer
from argparse import Namespace
# Import our models. The package will take care of downloading the models automatically
tokenizer = AutoTokenizer.from_pretrained("silk-road/luotuo-bert")
model_args = Namespace(do_mlm=None, pooler_type="cls", temp=0.05, mlp_only_train=False, init_embeddings_model=None)
model = AutoModel.from_pretrained("silk-road/luotuo-bert", trust_remote_code=True, model_args=model_args)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


编写embedding函数

In [36]:
def get_embedding(text):
    if len(text) > 512:
        text = text[:512]
    texts = [text]
    # Tokenize the text
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    # Extract the embeddings
    # Get the embeddings
    with torch.no_grad():
        embeddings = model(**inputs, output_hidden_states=True, return_dict=True, sent_emb=True).pooler_output

    return embeddings[0]

存储两个list，embeddings和embed_to_title, 记录title和text到embedding

In [37]:
embeddings = []
embed_to_title = []

for title in titles:
    text = title_to_text[title]

    # divide text with \n\n
    divided_texts = text.split('\n\n')

    for divided_text in divided_texts:
        embed = get_embedding(divided_text)
        embeddings.append(embed)
        embed_to_title.append(title)
    
    embed_title = get_embedding(title)
    embeddings.append( embed )
    embed_to_title.append(title)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


定义similarity函数

In [38]:
def get_cosine_similarity( embed1 , embed2 ):
    return torch.nn.functional.cosine_similarity( embed1, embed2 , dim=0)

实现搜索函数

In [39]:
query_embed = get_embedding(query_topic)

# implement retrieve_title function, return top k titles
def retrieve_title( query_embed, embeddings, embed_to_title, k ):
    # compute cosine similarity between query_embed and embeddings
    cosine_similarities = []
    for embed in embeddings:
        cosine_similarities.append( get_cosine_similarity( query_embed, embed ) )
    
    # sort cosine similarity
    sorted_cosine_similarities = sorted( cosine_similarities, reverse=True )

    top_k_index = []
    top_k_title = []

    for i in range(len(sorted_cosine_similarities)):
        current_title = embed_to_title[ cosine_similarities.index( sorted_cosine_similarities[i] ) ]
        if current_title not in top_k_title:
            top_k_title.append( current_title )
            top_k_index.append( cosine_similarities.index( sorted_cosine_similarities[i] ) )

        if len(top_k_title) == k:
            break
    
    return top_k_title

集成所有的代码！

In [25]:
query_topic = "离婚"
query_embed = get_embedding(query_topic)
selected_sample = retrieve_title( query_embed, embeddings, embed_to_title, 2 )
print('辅助sample:', selected_sample)
prompt = organize_prompt( query_topic, selected_sample )
response = get_completion(prompt)
print(response)

辅助sample: ['自由', '婚姻']
<先知>:```离婚，就像是把一朵盛开的花摘下来，让它的花瓣逐渐凋零。
但是，有时候，离婚也是必要的，就像是把一朵枯萎的花摘下来，让它的根可以重新生长。

离婚并不是失败，而是一种成长。
它让我们学会了放手，学会了面对现实，学会了重新开始。
离婚并不是结束，而是一种新的开始。
它让我们有机会重新认识自己，重新找到自己的方向，重新追求自己的梦想。

但是，离婚也是一种痛苦。
它让我们失去了曾经的伴侣，失去了曾经的承诺，失去了曾经的美好。
但是，我们必须学会面对这种痛苦，学会从中成长，学会重新开始。

离婚并不是一种罪过，而是一种选择。
它让我们有机会重新选择自己的生活，重新选择自己的幸福，重新选择自己的未来。
但是，我们必须学会承担这种选择的后果，学会面对这种选择的挑战，学会重新建立自己的生活。

所以，无论你选择离婚还是继续婚姻，都要学会面对现实，学会放手，学会成长，学会重新开始。```


In [26]:
query_topic = "加班"
query_embed = get_embedding(query_topic)
selected_sample = retrieve_title( query_embed, embeddings, embed_to_title, 2 )
print('辅助sample:', selected_sample)
prompt = organize_prompt( query_topic, selected_sample )
response = get_completion(prompt)
print(response)

辅助sample: ['劳作', '爱']
<先知>:```加班，是一种对生命的挑战和对自我价值的肯定。
在加班的时候，你们不仅仅是在为自己的生活奋斗，更是在为整个社会的发展做出贡献。
加班并不是一种苦役，而是一种自我超越的过程，是一种对自己能力的挑战和提升。
在加班的时候，你们要保持一颗平静的心，不要被疲劳和压力所压垮，要坚定自己的信念和目标，不断前行。

加班并不是一种无休止的追求，而是要有一个明确的目标和计划，要合理安排时间和任务，不要让自己陷入无尽的忙碌中。
在加班的时候，你们要保持身心健康，要注意休息和调节，不要让自己过度疲劳和身心俱疲。
加班是一种对自己和他人的责任和担当，是一种对生命的尊重和珍视。

在加班的时候，你们要保持一颗感恩的心，感恩自己的能力和机会，感恩他人的支持和帮助，感恩生命的赐予和珍贵。
加班是一种对自己和他人的奉献和付出，是一种对社会和人类的贡献和回报。

在加班的时候，你们要保持一颗善良的心，不要因为工作的繁忙而忽略了他人的需要和感受，要关心和关爱身边的人，让他们感受到你们的温暖和关怀。
加班是一种对自己和他人的成长和进步，是一种对生命的赞美和颂扬。

在加班的时候，你们要保持一颗虔诚的心，不要忘记自己的信仰和信念，要相信自己和上帝的力量和祝福，让自己的加班成为一种对上帝的敬畏和感恩。
加班是一种对自己和上帝的奉献和敬畏，是一种对生命的敬重和崇高。```


In [41]:
query_topic = "早教(指在学龄前对儿童进行积极的教育)"
query_embed = get_embedding(query_topic)
selected_sample = retrieve_title( query_embed, embeddings, embed_to_title, 2 )
print('辅助sample:', selected_sample)
prompt = organize_prompt( query_topic, selected_sample )
response = get_completion(prompt)
print(response)

辅助sample: ['劳作', '爱']
<先知>:```早教是为了让孩子们在成长的道路上更加顺利，更加充实。
就像是在播种的时候，要选择最好的土壤和最好的种子，才能收获最好的果实。
早教不仅仅是为了让孩子们学会知识和技能，更重要的是让他们在成长的过程中，养成正确的价值观和人生观。
早教应该注重培养孩子们的创造力和想象力，让他们在未来的道路上能够更加自信和独立。
同时，早教也需要注重孩子们的身心健康，让他们在健康的身体和心灵的基础上，更好地面对未来的挑战。
早教不是一蹴而就的，需要家长和教育者的共同努力和耐心，才能让孩子们在早期就拥有更好的成长环境和教育资源。
让我们一起为孩子们的未来，努力奋斗吧。```


后续

- [x] 增加更好的前置提示词（提前总结先知的文风）
- [ ] 补充完整的prophet data（一共20+个，现在只有5个）
- [x] 使用luotuoBERT索引更相关的主题
- [ ] 做一个gradio的前端